# pipeline test

This is a text cell. Start editing!

In [1]:
import com.johnsnowlabs.nlp.SparkNLP
import com.johnsnowlabs.nlp.annotator._
import com.johnsnowlabs.nlp.base._
import com.johnsnowlabs.ml.tensorflow.TensorflowBert
import org.apache.spark.ml.Pipeline
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions.{udf,to_timestamp}
import org.apache.spark.ml.feature.QuantileDiscretizer
import org.apache.spark.storage._

val dataDir = "../../data/recsys2020"

In [2]:
// val df = spark.read.parquet(dataDir + "/training1m.parquet").limit(1000).persist(StorageLevel.MEMORY_ONLY) // Remove limit after experiments
// df

In [3]:
// Convert targets
// val udf_has_engagement = udf[Int, IntegerType](x => if (x != null) 1 else 0)
// var trans_df = df.withColumn("has_retweet", udf_has_engagement('retweet_timestamp))
// trans_df

In [4]:
val trainDf = Seq(0 to 100:_*).toDF().withColumn("text", lit("Hello World"))
    .withColumn("target1", when(rand() > 0.3, 1).otherwise(0))
    .withColumn("target2", when(rand() > 0.6, 1).otherwise(0))
trainDf.show()

+-----+-----------+-------+-------+
|value|       text|target1|target2|
+-----+-----------+-------+-------+
|    0|Hello World|      0|      1|
|    1|Hello World|      1|      0|
|    2|Hello World|      1|      1|
|    3|Hello World|      1|      0|
|    4|Hello World|      1|      1|
|    5|Hello World|      1|      0|
|    6|Hello World|      1|      1|
|    7|Hello World|      1|      1|
|    8|Hello World|      1|      0|
|    9|Hello World|      0|      0|
|   10|Hello World|      1|      0|
|   11|Hello World|      0|      0|
|   12|Hello World|      0|      1|
|   13|Hello World|      1|      1|
|   14|Hello World|      1|      1|
|   15|Hello World|      1|      0|
|   16|Hello World|      1|      0|
|   17|Hello World|      1|      1|
|   18|Hello World|      0|      1|
|   19|Hello World|      0|      0|
+-----+-----------+-------+-------+
only showing top 20 rows



In [5]:
val doc = new DocumentAssembler()
    .setInputCol("text")
    .setOutputCol("document")
    .setCleanupMode("shrink")
// val tok = new Tokenizer()
//     .setInputCols("document")
//     .setOutputCol("token")
//     .setContextChars(Array("(", ")", "?", "!"))
//     .setSplitChars(Array("-"))
//     .addException("New York")
//     .addException("e-mail")
// val bert = BertEmbeddings.pretrained(name="bert_multi_cased", lang="xx")
//       .setInputCols("document", "token")
//       .setOutputCol("embeddings")
//       .setPoolingLayer(0) // 0, -1, or -2
val use = UniversalSentenceEncoder
      .pretrained()
      .setInputCols(Array("document"))
      .setOutputCol("tweet_embeddings")
// val emb = new SentenceEmbeddings()
//       .setInputCols(Array("document", "embeddings"))
//       .setOutputCol("tweet_embeddings")
//       .setPoolingStrategy("AVERAGE")
val fin = new EmbeddingsFinisher()
      .setInputCols("tweet_embeddings")
      .setOutputCols("finished_tweet_embeddings")
      .setOutputAsVector(true)
      .setCleanAnnotations(false)

// val use_clf_pipeline = new Pipeline().setStages(Array(doc, tok, bert, emb, fin))
val use_clf_pipeline = new Pipeline().setStages(Array(doc, use, fin))

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
Download done! Loading the resource.


In [6]:
use_clf_pipeline.fit(testDf).transform(testDf)